## *Extract , Transform & Load – Superstore Dataset*

*Author: Mohamed (ETL Lead) – Group 8*

 ### *Importing necessary Libraries*

In [43]:
import pandas as pd
import numpy as numpy
import sqlite3
import os

### *Defining Paths so as to make it easy to reference them*

In [59]:
RAW_PATH = "data/superstore_sales_data.csv"
TRANSFORMED_PATH = "output/superstore_transformed.csv"
FINAL_CSV_PATH = "output/superstore_final.csv"
FINAL_DB_PATH = "output/superstore_final.db"
TABLE_NAME = "superstore_sales"

### *Extract and Inspecting Dataset*

In [53]:
print("Extracting raw data...")
superstore_data = pd.read_csv(RAW_PATH,encoding='ISO-8859-1', index_col="Row ID")
superstore_data.head()

Extracting raw data...


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
Row ID,,,,,,,,,,,,,,,,,,,,
1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [60]:
# Dataset overview
print("\nInitial Overview:")
print("Shape of dataset:", superstore_data.shape)


Initial Overview:
Shape of dataset: (9993, 20)


### *Dataset Metadata – Superstore Sales*

- `Row ID` – *Unique ID for each row.*
- `Order ID` – *Unique Order ID for each customer.*
- `Order Date` – *Date when the product was ordered.*
- `Ship Date` – *Date when the product was shipped.*
- `Ship Mode` – *Shipping mode selected by the customer.*
- `Customer ID` – *Unique ID used to identify each customer.*
- `Customer Name` – *Name of the customer.*
- `Segment` – *Customer segment (e.g., Consumer, Corporate, Home Office).*
- `Country` – *Country where the customer resides.*
- `City` – *City where the customer resides.*
- `State` – *State where the customer resides.*
- `Postal Code` – *Postal code of the customer's address.*
- `Region` – *Region the customer belongs to (e.g., East, West).*
- `Product ID` – *Unique identifier for the product.*
- `Category` – *Category of the product (e.g., Furniture, Technology).*
- `Sub-Category` – *Sub-category of the product.*
- `Product Name` – *Name or title of the product ordered.*
- `Sales` – *Total sales value of the product.*
- `Quantity` – *Number of product units ordered.*
- `Discount` – *Discount applied on the product.*
- `Profit` – *Profit or loss incurred from the sale.*



In [52]:
superstore_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

#### *Structure Overview*

*After loading the dataset, we use `.info()` to understand the structure:*

- *The dataset contains **9,994 rows** and **21 columns**.*
- *All columns have **complete data** — no missing values.*
- *The data types show a mix of:*
  - ***Object** types for categorical and string data*
  - ***Integer** for quantities like Postal Code and Quantity*
  - ***Float** for Sales, Discount, and Profit*


### *Data_Cleaning*
in this code i am cleaning the dataset by looking for any missing values, duplicates and then droping them. I also renamed the columns.
 

In [55]:
#checking for d+uplicates
superstore_data.duplicated().sum()

0

In [56]:
# Remove duplicate row
superstore_data.drop_duplicates(inplace=True)

In [57]:
#checking for null values
print(superstore_data.isnull().sum())

Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64


In [58]:
# Drop duplicate rows
superstore_data.drop_duplicates(inplace=True)

# Convert 'Order Date' and 'Ship Date' to datetime
superstore_data['Order Date'] = pd.to_datetime(superstore_data['Order Date'])
superstore_data['Ship Date'] = pd.to_datetime(superstore_data['Ship Date'])

# Convert selected object columns to category
categorical_cols = [
    "Ship Mode", "Segment", "Region", "Category", "Sub-Category", "State"
]

for col in categorical_cols:
    superstore_data[col] = superstore_data[col].astype("category")


In [ ]:
# df = df_raw.copy()
# df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("-", "_")

# print("\nMissing Values Summary:")
# print(df.isnull().sum())


# df.dropna(subset=["Order_ID", "Order_Date", "Sales", "Profit"], inplace=True)


# initial_shape = df.shape
# df.drop_duplicates(inplace=True)
# print(f"\nRemoved {initial_shape[0] - df.shape[0]} duplicate rows.")


Missing Values Summary:
Row_ID           0
Order_ID         0
Order_Date       0
Ship_Date        0
Ship_Mode        0
Customer_ID      0
Customer_Name    0
Segment          0
Country          0
City             0
State            0
Postal_Code      0
Region           0
Product_ID       0
Category         0
Sub_Category     0
Product_Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

Removed 0 duplicate rows.


### *Transformation*

Now I apply the following transformations to improve data quality and structure:
1. **Date Conversion**: Convert `Order_Date` and `Ship_Date` to datetime objects.
2. **Numeric Conversion**: Ensure columns like `Sales`, `Profit`, `Quantity`, and `Discount` are treated as numeric types.
3. **Text Standardization**: Standardize text fields such as `Customer_Name`, `City`, and `State` using `.str.title()`.
4. **Enrichment**: I create a new column called `Profit_Margin` by dividing `Profit` by `Sales`, which helps in later if we do financial analysis.

These transformations will help downstream processes like mining and visualization.


In [41]:

df["Order_Date"] = pd.to_datetime(df["Order_Date"], errors="coerce")
df["Ship_Date"] = pd.to_datetime(df["Ship_Date"], errors="coerce")

numeric_cols = ["Sales", "Quantity", "Discount", "Profit"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

df["Customer_Name"] = df["Customer_Name"].str.title()
df["City"] = df["City"].str.title()
df["State"] = df["State"].str.title()

df["Profit_Margin"] = (df["Profit"] / df["Sales"]).round(2).replace([float('inf'), -float('inf')], pd.NA)

df.to_csv(TRANSFORMED_PATH, index=False)
print(f"\nCleaned data saved to {TRANSFORMED_PATH}")


Cleaned data saved to data/superstore_transformed.csv


### *loading the dataset*
in this cell i load the datset using sqlite3 and save the db file.

In [42]:
conn = sqlite3.connect(FINAL_DB_PATH)
df.to_sql(TABLE_NAME, conn, if_exists="replace", index=False)
conn.close()

print(f"SQLite DB saved to {FINAL_DB_PATH}")

print("\nETL pipeline complete.")

SQLite DB saved to data/superstore_final.db

ETL pipeline complete.
